<a href="https://colab.research.google.com/github/pranaya-mathur/Deep-Leaning/blob/master/Toxic_Comment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chinki86","key":"2b37558d5b359235a655730a30d3156b"}'}

In [4]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!pip install kaggle

!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge



-rw-r--r-- 1 root root 64 Dec 26 14:29 kaggle.json
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 45.0MB/s]
 38% 9.00M/23.4M [00:00<00:00, 81.6MB/s]
100% 23.4M/23.4M [00:00<00:00, 115MB/s] 
 87% 23.0M/26.3M [00:00<00:00, 66.0MB/s]
100% 26.3M/26.3M [00:00<00:00, 104MB/s] 
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 97.3MB/s]


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("glove.6B.zip", 'r')
zip_ref.extractall("glove")
zip_ref.close()

In [0]:
import pandas as pd
import numpy as np

In [0]:
import os
import sys
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalAveragePooling1D, Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score

In [11]:
! wget https://nlp.stanford.edu/data/glove.6B.zip

--2018-12-26 15:33:22--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  87.7MB/s    in 9.8s    

2018-12-26 15:33:32 (84.2 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [15]:
word2vec = {}
with open("glove/glove.6B.100d.txt") as f:
  
  for line in f:
    
    values = line.split()
    word = values[0]
    
    vec = np.asarray(values[1:],dtype="float32")
    word2vec[word] = vec
    
print("Found %s word vectors." % len(word2vec))
  

Found 400000 word vectors.


In [18]:
word2vec["the"].shape

(100,)

In [19]:
train = pd.read_csv("Train/train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [20]:
sentences = train["comment_text"].fillna("DUMMY_VALUES").values
sentences

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [21]:
sentences.shape

(159571,)

In [0]:
possible_labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

In [0]:
targets = train[possible_labels].values

In [24]:
print("max sequence length:", max(len(s) for s in sentences))
print("min sequence length:", min(len(s) for s in sentences))

max sequence length: 5000
min sequence length: 6


In [0]:
s = sorted(len(s) for s in sentences)

In [0]:
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE)

In [0]:
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [0]:
# get word -> integer mapping
word2idx = tokenizer.word_index

In [34]:
word2idx

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'have': 18,
 'are': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'wikipedia': 28,
 'my': 29,
 'an': 30,
 'from': 31,
 'by': 32,
 'do': 33,
 'at': 34,
 'me': 35,
 'about': 36,
 'so': 37,
 'talk': 38,
 'what': 39,
 'can': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'will': 44,
 'please': 45,
 'no': 46,
 'would': 47,
 'one': 48,
 'like': 49,
 'just': 50,
 'they': 51,
 'he': 52,
 'which': 53,
 'any': 54,
 'been': 55,
 'should': 56,
 'more': 57,
 'we': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'here': 63,
 'see': 64,
 'also': 65,
 'his': 66,
 'think': 67,
 'because': 68,
 'know': 69,
 'how': 70,
 'edit': 71,
 'am': 72,
 "i'm": 73,
 'people': 74,
 'why': 75,
 'up': 76,
 'only': 77,
 "it's": 78,
 'out': 79,
 'articles': 80,
 'use': 81,

In [35]:
len(word2idx)

210337

In [0]:
# pad sequences so that we get N x T matrix

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [37]:
data.shape

(159571, 100)

In [38]:
data

array([[    0,     0,     0, ...,  4583,  2273,   985],
       [    0,     0,     0, ...,   589,  8377,   182],
       [    0,     0,     0, ...,     1,   737,   468],
       ...,
       [    0,     0,     0, ...,  3509, 13675,  4528],
       [    0,     0,     0, ...,   151,    34,    11],
       [    0,     0,     0, ...,  1627,  2056,    88]], dtype=int32)

In [39]:
# prepare our embedding matrix

num_words = min(MAX_VOCAB_SIZE,len(word2idx)+1)
num_words

20000

In [0]:
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))

In [0]:
for word, i in word2idx.items():
  
  if i < MAX_VOCAB_SIZE:
    
    
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      
      embedding_matrix[i] = embedding_vector

In [43]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.18970001,  0.050024  ,  0.19084001, ..., -0.39804   ,
         0.47646999, -0.15983   ],
       ...,
       [-1.04809999, -0.32789999, -0.46593001, ..., -0.065447  ,
        -0.13019   ,  0.44433001],
       [ 0.089782  , -0.33164999, -0.85716999, ..., -0.15765999,
        -0.58174002,  0.036233  ],
       [ 0.051446  , -0.43564001,  0.26989001, ...,  0.39087   ,
        -0.14459001, -0.72372001]])

In [0]:
embedding_layer = Embedding(num_words,EMBEDDING_DIM,
                           weights = [embedding_matrix],
                           input_length = MAX_SEQUENCE_LENGTH,
                           trainable=False)

In [47]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128,3,activation="relu")(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128,3,activation="relu")(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128,3,activation="relu")(x)
x = GlobalAveragePooling1D()(x)
x = Dense(128,activation="relu")(x)

output = Dense(len(possible_labels),activation="sigmoid")(x)

model = Model(input_,output)

model.compile(loss="binary_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

r = model.fit(data,targets,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=VALIDATION_SPLIT)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 15s 121us/step - loss: 0.0873 - acc: 0.9723 - val_loss: 0.0810 - val_acc: 0.9751
Epoch 2/10
127656/127656 [==============================] - 12s 90us/step - loss: 0.0682 - acc: 0.9772 - val_loss: 0.0699 - val_acc: 0.9766
Epoch 3/10
127656/127656 [==============================] - 12s 90us/step - loss: 0.0639 - acc: 0.9783 - val_loss: 0.0738 - val_acc: 0.9767
Epoch 4/10
127656/127656 [==============================] - 11s 90us/step - loss: 0.0604 - acc: 0.9791 - val_loss: 0.0665 - val_acc: 0.9772
Epoch 5/10
127656/127656 [==============================] - 11s 90us/step - loss: 0.0576 - acc: 0.9797 - val_loss: 0.0680 - val_acc: 0.9770
Epoch 6/10
127656/127656 [==============================] - 11s 90us/step - loss: 0.0549 - acc: 0.9804 - val_loss: 0.0688 - val_acc: 0.9780
Epoch 7/10
127656/127656 [==============================] - 11s 90us/step - loss: 0.0525 - acc: 0.9811 - val

In [0]:
p = model.predict(data)

In [49]:
aucs = []
for j in range(6):
  
  auc = roc_auc_score(targets[:,j],p[:,j])
  aucs.append(auc)
  
print(np.mean(aucs))

0.9754604795205433
